In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_groq import ChatGroq

In [34]:
import csv

csv_file_path = "Three_EPS_Req.csv"

def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements

In [41]:
requirements_doc = load_csv(csv_file_path)
requirements_doc

[{'id': '245112',
  'Primary Text': 'Power Supply',
  'Artifact Type': 'Heading',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': 'TRUE'},
 {'id': '245113',
  'Primary Text': "The Steering ECU shall ensure reliable operation within the specified voltage range of the vehicle electrical system while maintaining efficient power consumption under varying load conditions.\nOperating Range:\n• The Steering ECU shall operate on a 12V or 48V vehicle electrical system, with the following considerations:\n    '- Nominal Voltage: 12V system.\n    '- Operating Voltage Range: 10V to 24V.\n• The ECU shall handle voltage fluctuations within this range without degradation in functionality or performance.",
  'Artifact Type': 'SYS Requirement',
  'Safety': 'ASIL A',
  'Verification_Criteria': 'The Steering ECU shall monitor its power supply and generate the following Diagnostic Trouble Codes (DTCs) when specific fault conditions occur: \no Trigger DTC_Voltage_Failure if the supply voltage

In [17]:
from PyPDF2 import PdfReader

# Specify the path to the PDF file
pdf_path = "EPS_Info.pdf"

# Create a PdfReader object
reader = PdfReader(pdf_path)

# Initialize a variable to store the text
pdf_text = ""

# Iterate through all pages and extract text
for page in reader.pages:
    pdf_text += page.extract_text()

# Print the extracted text
# print(pdf_text)

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " "]
)

requirement_texts = text_splitter.create_documents([pdf_text])
# sections = text_splitter.split_documents(texts)

# for i, section in enumerate(sections):
#     print(f"Section {i + 1}:\n{section}\n{'-' * 80}")

In [19]:
from langchain_core.documents import Document

documents = [(Document(page_content=str( {
            "Primary Text": item['Primary Text'],
            "Verification_Criteria": item['Verification_Criteria'],
        }), metadata={"Requirement ID": item["id"]})) for item in requirements_doc]

In [94]:
# import uuid
# from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings

# embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

# unique_collection_name = f"collection_{uuid.uuid4()}"

# client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# # Add documents
# vector_doc = client.add_documents(documents=documents, ids = ids)

In [20]:
import uuid
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ids = [str(uuid.uuid4()) for _ in range(len(requirement_texts))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

persist_directory = "chromadb_store"
vectorstore = Chroma.from_documents(requirement_texts, collection_name=unique_collection_name, embedding=embeddings_model, persist_directory=persist_directory)

print(f"Vector database saved in: {persist_directory}")

# Add documents
#client.add_documents(documents=requirement_texts, ids = ids)

Vector database saved in: chromadb_store


In [21]:
import re
from langchain.memory import ConversationBufferWindowMemory 
from langchain.chains import ConversationChain

def extract_xml(text: str, tag: str) -> str:
    """
    Extracts the content of the specified XML tag from the given text. Used for parsing structured responses 

    Args:
        text (str): The text containing the XML.
        tag (str): The XML tag to extract content from.

    Returns:
        str: The content of the specified XML tag, or an empty string if the tag is not found.
    """
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

# retrieving the relevant documents from vector store
def retrieve_documents(query_text, top_k=4):
    """Fetching relevant document from vector store """

    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model)
    
    retriever = vectorstore.as_retriever()
    retrieve_doc = retriever.invoke(query_text)
    
    # for item in retrieve_doc:
    #      documents.append({
    #             "content": item.page_content,
    #             "source": item.metadata,
    #         })  
    
    return retrieve_doc

#cleanising the retrieved vector store document
def clean_text(relevant_document):
    content_list = []
    for item in relevant_document:
        # Clean the 'content' field
        content = item['content']
        content = content.replace("'", '"')
        content = re.sub(r'[\'\-•]""', "", content)  # Remove specified characters
        content = re.sub(r'\s{2,}', ' ', content).strip()  # Normalize whitespace and trim
        item['content'] = content
        content_list.append(content)
        cleanised_document = ''.join(content_list)
    return cleanised_document

def llm_call_orches(prompt: str, system_prompt: str = "", model="deepseek-r1-distill-llama-70b") -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        model (str, optional): The model to use for the call. Defaults to "llama3-70b-8192".

    Returns:
        str: The response from the language model.
    """
    
    # messages = [{"role": "user", "content": prompt}]
    memory = ConversationBufferWindowMemory(k=10)
    
    llm = ChatGroq(
    model=model,
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
    )
    
    conversation = ConversationChain(
                    llm=llm,
                    memory=memory
    )
    response = conversation.invoke(prompt)
    #print(response)
    memory.save_context({"input": prompt}, {"response": response['response']})
    
    return response

In [31]:
from typing import Dict, List, Optional
from pprint import pprint
    
class Orchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""
    
    def __init__(self,orchestrator_prompt: str):
        self.orchestrator_prompt = orchestrator_prompt

    def _format_prompt(self, template: str,  **kwargs) -> str:
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process_llm(self, requirement: str, count: int) -> Dict:
        """Orchestrate tasks using the LLM based on the given requirement.
        
        Args:
            requirement (str): The requirement to process.
        
        Returns:
            str: Output from the orchestrator.
        """
        unpack = {}
        
        # retrieving and cleansing the document
        raw_documents = retrieve_documents(requirement)
        #documents = clean_text(raw_documents)
        
        if (count>0): 
            self.orchestrator_prompt = (f"{ORCHESTRATOR_PROMPT_MEMORY}")
            print("************ Understanding Requirement Again **************")
                        
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            requirement=requirement,
            documents=raw_documents,
            **unpack
        )
        orchestrator_response = llm_call_orches(orchestrator_input)
        #pprint(orchestrator_response)
        
        #analysis = extract_xml(orchestrator_response['response'], "think")
        #test_scenarios = extract_xml(orchestrator_response['response'], "tests")
        
        print("\n=== ORCHESTRATOR OUTPUT ===")
        # pprint(orchestrator_response)
        print(f"\nANALYSIS:\n{orchestrator_response['response']}")
        #print(f"\nTEST SCENARIOS:\n{test_scenarios}")
        
        return orchestrator_response['response']
        #return {"Analysis": analysis, "Test Scenarios": test_scenarios}

In [47]:
ORCHESTRATOR_PROMPT = """ You are provided with requirements of Automotive Electronic Control Unit(ECU). 
Analyze the requirement and understand the context of the it based on documents provided. You should understand the requirement completely.
        requirement: {requirement}
        documents: {documents}

Generate test scenarios to validate the requirement. if you have any doubt please ask for clarifications and don't generate test scenarios
"""

WORKER_PROMPT = """ You are a test case generation assistant who will create test cases for given Test Scenarios. You are well aware of software testing and its testing techniques.
Requirement context is given to you as input, your task is to understand the requirement context and generate the test cases with different testing techniques in order to achieve maximum test coverage of requirement.

Requirements context:{context_requirement} 

Output your answer concisely in the following format, with "NO PREMABLE": 

<thoughts>
[Your understanding of the requirement and how you planned to write test cases]
</thoughts>

<test_cases>
Test Case ID : ID for test case (should be whole number)
Objective : Objective of test case
Test Design Technique : Test design technique used for this test
Pre-conditions : What are the preconditions before performing the test
Post-conditions:  What are the post conditions after performing the test
Steps to Execute : Detailed test steps
Expected Results : Expected outcome for test step
</test_cases>

Note: Measure that test case ID should be only integer number and not decimal numbers.
"""

EVAL_PROMPT = """ You are expert in software testing and well aware testing standards. You are provided with certain test cases and its corresponding requirement context 
Your task is to evaluate:
- Whether the test cases achieve the maximum coverage for input requirement context.
- Whether the test cases are written as per standard testing guidelines.
- Whether the test steps and expected results are clear and concise

Output "PASS" only if you don't have any further improvement points based on above evaluations.
Output "SATISFACTORY" if the test cases satisfy the above evaluations.

Note: You should evaluate and specify improvement points only based on requirement context and provided criteria, don't makeup your own answer.

Output your evaluation concisely in the following format.
<evaluation>
Feedback level: PASS, SATISFACTORY, NEEDS_IMPROVEMENT, or FAIL.
Explain you plan to evaluate the test cases and your understanding of requirement context
</evaluation>

<feedback>
Evaulate and provide the improvement points for test cases, if any. Please be specific. Mention if more test cases needs to be generated along with test sceanrios.
</feedback>

Here are the inputs you need to perform task on
Requirement Context: {Understanding}
Test cases to evaluate: {test_cases}
"""

FEEDBACK_PROMPT_WORKER = """ Based on the provided feedback Rewrite the test cases and Output the response in below format. 
Note: While generating new test cases based on feedback, include the previous test cases also.

<thoughts>
[Your understanding of the feedback and how you plan to improve]
</thoughts>

<test_cases>
Test Case ID : ID for test case (should be whole number)
Objective : Objective of test case
Test Design Technique : Test design technique used for this test
Pre-conditions : What are the preconditions before performing the test
Post-conditions:  What are the post conditions after performing the test
Steps to Execute : Detailed test steps
Expected Results : Expected outcome for test step 
</test_cases>
"""
FEEDBACK_PROMPT_EVAL = """ Based on the provide feedback, the test cases has been modified. Please evaluate again.

Output "PASS" if the test cases satisfy the above evaluations completely.

Output your evaluation concisely in the following format.
<evaluation
PASS, SATISFACTORY, NEEDS_IMPROVEMENT, or FAIL.
</evaluation>

<feedback>
Evaulate and provide the improvement points for test cases, if any. Please be specific.
Specify if more test cases needs to be generated along with test sceanrios.
</feedback>
"""
ORCHESTRATOR_PROMPT_MEMORY = """Now Understand these requirements and provide the response.
requirement: {requirement}
documents: {documents}

Return your response in this format:

<analysis>
Explain your understanding of the given requirement in detail. Break down it in detail. Don't include any "PREAMBLE"
</analysis>

"""

In [48]:
sub_req = requirements_doc[3:6]

In [49]:
orchestrator = Orchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT
)
requirement_count=0
for item in requirements_doc:
    if item['Artifact Type'] == 'SYS Requirement':
        print(f"*********** Understanding context of Requirement {item['id']} and generating Test scenarios to validate requirement **************")
        result_analysis = orchestrator.process_llm(requirement=item['Primary Text'], count=requirement_count)
        #result = feedback_loop(result_analysis, EVAL_PROMPT, WORKER_PROMPT)
        #final_result.append(feedback_loop(result_analysis, EVAL_PROMPT, WORKER_PROMPT))
        #requirement_count=requirement_count+1

*********** Understanding context of Requirement 245113 and generating Test scenarios to validate requirement **************

=== ORCHESTRATOR OUTPUT ===

ANALYSIS:
<think>
Okay, so I need to generate test scenarios to validate the given requirement for the Steering ECU. The requirement is that the ECU must operate reliably within a specified voltage range while maintaining efficient power consumption under varying loads. The operating range is 10V to 24V for a 12V nominal system, and it should handle fluctuations without any degradation.

First, I should understand what each part of the requirement entails. The ECU needs to work reliably, which means it shouldn't malfunction or have errors when the voltage is within the specified range. It also needs to maintain efficient power consumption, so even when the voltage changes, the ECU shouldn't draw more power than necessary, which could drain the vehicle's battery or cause overheating.

The operating voltage range is from 10V to 24V. Th

In [12]:
import os
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser

unpack={}

llm_generator = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

llm_eval = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

def _format_prompt_worker(template: str,  **kwargs) -> str:
    """Format a prompt template with variables."""
    try:
        return template.format(**kwargs)
    except KeyError as e:
        raise ValueError(f"Missing required prompt variable: {e}")

In [37]:
def generate_test_case(context: str, prompt: str, feedback: str = "") -> tuple[str, str]:
    
    # full_prompt = f"{prompt}\n{feedback}\n" if feedback else f"{prompt}\n"
    
    print(f"*************** Generating Test case **********************")
    
    memory_generate = ConversationBufferWindowMemory(k=10)
    conversation_generate = ConversationChain(
                    llm=llm_generator,
                    memory=memory_generate
    )
    # raw_documents = retrieve_documents(context,client)
    # documents = clean_text(raw_documents)
    
    if feedback:
        worker_input = (f"{feedback}\n{FEEDBACK_PROMPT_WORKER}")
        print("************ Rewriting test cases as per feedback provided **************")
    else:
        full_prompt = f"{prompt}\n"
        worker_input = _format_prompt_worker(
                    full_prompt,
                    context_requirement=context,
                    **unpack
        )
    
    #pprint(f"******************* TC generation prompt\n{worker_input}\n")
        
    worker_response = conversation_generate.invoke(worker_input)
    #pprint(worker_response)
    thoughts = extract_xml(worker_response['response'], "thoughts") 
    test_cases = extract_xml(worker_response['response'], "test_cases")
    print(f"====== Thought ======\n{thoughts}")
    print(f"====== Test cases====\n{test_cases}\n")
    memory_generate.save_context({"thoughts": feedback}, {"test_cases": test_cases})

    return test_cases

In [40]:
def evaluate_test_case(context: str, prompt: str, test_cases: str, feedback: str = "") -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    
    memory_eval = ConversationBufferWindowMemory(k=10)
    conversation_eval = ConversationChain(
                    llm=llm_eval,
                    memory=memory_eval
    )
    if feedback:
        eval_input = (f"{test_cases}\n{FEEDBACK_PROMPT_EVAL}")
        print("************ Evaluating test cases Again **************")
    
    else:
        full_prompt = f"{prompt}\n"
        print("************** Evaluating Test Case ****************")
        eval_input = _format_prompt_worker(
                    full_prompt,
                    Understanding=context,
                    test_cases=test_cases,
                    **unpack
        )
    #pprint(f"******************* TC evaluation prompt\n{eval_input}\n")
    
    eval_response = conversation_eval.invoke(eval_input)
    evaluation = extract_xml(eval_response['response'], "evaluation")
    feedback = extract_xml(eval_response['response'], "feedback")
    #pprint(worker_response)
              
    print(f"\n=== Eval RESULT ===\n{evaluation}\n{feedback}")
    memory_eval.save_context({"test cases": test_cases}, {"feedback": feedback})
    return evaluation, feedback
    

In [47]:
def feedback_loop(context: str, evaluator_prompt: str, generator_prompt: str) -> list[dict]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    test_cases = generate_test_case(context, generator_prompt)
    memory.append(test_cases)
    feedback= ""
    
    max_iteration = 4
    for i in range(max_iteration):
        #print(f"\n========================= iteration for evaluation {i+1} ==========================\n")
        evaluation, feedback = evaluate_test_case(context, evaluator_prompt, test_cases, feedback=feedback)
        if evaluation.strip() == "PASS":
            test_cases = [iterator_str.strip() for iterator_str in test_cases.split("\n\n") if iterator_str.strip()]
            print(test_cases)
            return test_cases
            
        # feedback = "\n".join([*[f"- {m}" for m in memory],
        #     f"\nFeedback: {feedback}"
        # ])
        #print(context_join)
        
        test_cases = generate_test_case(context, generator_prompt, feedback=feedback)
        memory.append(test_cases)
        chain_of_thought.append({"test cases": test_cases})
    return chain_of_thought

In [11]:
orchestrator = Orchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT
)

requirement_count = 0
final_result = []
for item in sub_req:
    if item['Artifact Type'] == 'SYS Requirement':
        print(f"*********** Understanding context of Requirement {item['id']} and generating Test scenarios to validate requirement **************")
        result_analysis = orchestrator.process_llm(requirement=item['Primary Text'], count=requirement_count)
        #result = feedback_loop(result_analysis, EVAL_PROMPT, WORKER_PROMPT)
        final_result.append(feedback_loop(result_analysis, EVAL_PROMPT, WORKER_PROMPT))
        requirement_count=requirement_count+1

NameError: name 'Orchestrator' is not defined

In [83]:
final_result

[["Test Case ID: 1\nObjective: To verify the ECU's output for different steering wheel angle inputs within the effective range (±180°)\nTest Design Technique: Equivalence Partitioning\nPre-conditions: ECU is initialized and calibrated\nPost-conditions: ECU's output is within the expected range\nSteps to Execute:\n1. Set steering wheel angle to -180° and measure ECU's output\n2. Set steering wheel angle to -90° and measure ECU's output\n3. Set steering wheel angle to 0° and measure ECU's output\n4. Set steering wheel angle to 90° and measure ECU's output\n5. Set steering wheel angle to 180° and measure ECU's output\nExpected Results: ECU's output is within the expected range for each steering wheel angle input",
  "Test Case ID: 2\nObjective: To verify the ECU's output for boundary values of steering wheel angle\nTest Design Technique: Boundary Value Analysis\nPre-conditions: ECU is initialized and calibrated\nPost-conditions: ECU's output is within the expected range\nSteps to Execute:

In [84]:
    # for item in final_result:
    #     requirement_covered = item.get("Requirement Covered", "N/A")  # Default to "N/A" if key is missing
    #     buffer_testcase = item.get("Test Cases", "")
for set_of_cases in final_result:
     for case in set_of_cases:
        lines = [line.strip() for line in case.split("\n") if line.strip()]
        test_case_id = next(
            (line.split(":", 1)[1].strip() for line in lines if line.startswith("Test Case ID")), "N/A")
        objective = next((line.split(":", 1)[1].strip() for line in lines if line.startswith("Objective")),
                         "N/A")
        design_technique = next(
            (line.split(":", 1)[1].strip() for line in lines if line.startswith("Test Design Technique")),
            "N/A")
        pre_conditions = next(
            (line.split(":", 1)[1].strip() for line in lines if line.startswith("Pre-conditions")), "N/A")
        post_conditions = next(
            (line.split(":", 1)[1].strip() for line in lines if line.startswith("Post-conditions")), "N/A")
        steps_start = next((i for i, line in enumerate(lines) if line.startswith("Steps to Execute")),
                           len(lines))
        results_start = next((i for i, line in enumerate(lines) if line.startswith("Expected Results")),
                             len(lines))
    
        steps = "\n".join(lines[steps_start + 1:results_start]).strip() if steps_start < len(lines) else "N/A"
        expected_results = next(
            (line.split(":", 1)[1].strip() for line in lines if line.startswith("Expected Results")), "N/A")
    
        print(f"Test case ID: {test_case_id}\n Expected results: {expected_results}\n")

Test case ID: 1
 Expected results: ECU's output is within the expected range for each steering wheel angle input

Test case ID: 2
 Expected results: ECU's output is within the expected range for in-range inputs and errors out for out-of-range inputs

Test case ID: 3
 Expected results: ECU's output is within the expected range for each steering torque input value

Test case ID: 4
 Expected results: ECU's output is within the expected range for each turning radius input value

Test case ID: 5
 Expected results: ECU's output is within the expected range for each vehicle speed input value

Test case ID: 6
 Expected results: ECU's output is within the expected range for each combination of inputs

Test case ID: 7
 Expected results: ECU's output is within the expected range for each combination of inputs

Test case ID: 8
 Expected results: ECU's output is an error message for invalid inputs and within the expected range for valid inputs

Test case ID: 1
 Expected results: Assistive torque is

In [21]:
!pip show langchain

Name: langchain
Version: 0.3.13
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\LJO2KOR\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: embedchain, langchain-community, langchain-tools
